# Info games enrichment

This script allows to enrich the `games_datasets` with games info from SteamDB

## PreProcessing

In [1]:
#!pip install thefuzz
#!pip install python-Levenshtein

In [2]:
import pandas as pd
import json
import collections
import re
import numpy as np
import requests
from thefuzz import fuzz
from thefuzz import process

C:\ProgramData\Anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Import Twitch games

In [3]:
games_tw = pd.read_csv('Output_datasets/Games_dataset_name.csv', index_col='ID_game')
games_tw

,game_name
ID_game,
0,Monster Hunter Generations Ultimate
1,Alice: Madness Returns
2,Hold Fast
3,Pokémon Trading Card Game Online
4,Witch It
...,...
1139,Rise of Nations: Extended Edition
1140,Paper Mario: The Origami King
1141,The Inheritance of Crimson Manor


#### Clean strings

Create dictionary with:

    {'ID_game' : { \
         'twitch_name': ' ', \
         'twitch_clean': ' ', \
         'steam_name' : ' ', \
         'steam_clean' : ' ', \
         'match_score': ' '\
         }}
         
where `twitch_clean` and `steam_clean` are:

    -strip
    -lower
    -alphanumeric + èòàìùüé
    -remove multiple spaces
    -remove all duplicates

In [4]:
match_dict = {}
clean = ''

for i in range(len(games_tw)):
    match_dict[i] = {}
    match_dict[i]['twitch_name'] = games_tw.game_name[i]
    clean = re.sub(r"[^a-zA-Z0-9èòàìùüé ']",'',games_tw.game_name[i]).lower().strip()
    clean = re.sub('\\s+', ' ', clean)
    match_dict[i]['twitch_clean'] = clean
    match_dict[i]['steam_name'] = ''
    match_dict[i]['steam_clean'] = ''
    match_dict[i]['match_score'] = ''
len(match_dict)

1144

In [5]:
match_df = pd.DataFrame.from_dict(match_dict,orient='index')
match_df

,twitch_name,twitch_clean,steam_name,steam_clean,match_score
0,Monster Hunter Generations Ultimate,monster hunter generations ultimate,,,
1,Alice: Madness Returns,alice madness returns,,,
2,Hold Fast,hold fast,,,
3,Pokémon Trading Card Game Online,pokémon trading card game online,,,
4,Witch It,witch it,,,
...,...,...,...,...,...
1139,Rise of Nations: Extended Edition,rise of nations extended edition,,,
1140,Paper Mario: The Origami King,paper mario the origami king,,,
1141,The Inheritance of Crimson Manor,the inheritance of crimson manor,,,
1142,Occult,occult,,,


### Import Steam games

Reading the data from the file

In [6]:
with open('../DataCollection/Steam_games/steam_games.json', 'r') as f:
    games = json.load(f)

In [7]:
games['1599340'].keys()

dict_keys(['title', 'Developer', 'Publisher', 'Release Date', 'Last Update', '24h Player Peak', 'Followers', 'categories'])

#### Remove duplicates

In [8]:
steam_games = []

for keys in games:
    steam_games.append(games[keys]['title'])
print(len(steam_games))
print(len(set(steam_games)))

3498
3488


In [9]:
dupl_games =[item for item, count in collections.Counter(list(steam_games)).items() if count > 1]
dupl_games

["Conqueror's Blade",
 'DEAD OR ALIVE Xtreme Venus Vacation',
 'Fallout: New Vegas',
 'Ironsight',
 'Romance of the Three Kingdoms Hadou',
 'Wolfenstein II: The New Colossus',
 'Dishonored',
 'Wolfenstein: The New Order',
 'Anomaly Zone',
 'GRID']

In [10]:
keys_dupl = []
for elem in dupl_games:
    for key in games:
        if elem == games[key]['title']:
            keys_dupl.append(key)

Elimino i 10 duplicati presenti nella lista steam_games: dovrò poi risalire alle informazioni del json per capire in caso di match quali informazioni associare al gioco

In [11]:
steam_games = list(set(steam_games))

#### Clean strings

steam_clean ottenuta con:

    -strip
    -lower
    -alphanumeric + èòàìùüé
    -remove multiple spaces
    -remove all duplicates

In [12]:
steam_dict = {}

for i in range(len(steam_games)):
    steam_dict[i] = {}
    steam_dict[i]['steam_name'] = steam_games[i]
    clean = re.sub(r"[^a-zA-Z0-9èòàìùüé ']",'',steam_games[i]).lower().strip()
    clean = re.sub('\\s+', ' ', clean)
    steam_dict[i]['steam_clean'] = clean
len(steam_dict)

3488

In [13]:
steam_df = pd.DataFrame.from_dict(steam_dict,orient='index')
steam_df.head()

,steam_name,steam_clean
0,EA SPORTS™ FIFA 21,ea sports fifa 21
1,RoBoRumble,roborumble
2,Anomaly 2,anomaly 2
3,Stealth Inc 2: A Game of Clones,stealth inc 2 a game of clones
4,Banished,banished


In [14]:
steam_df[steam_df.steam_clean == '']

,steam_name,steam_clean
37,觅长生,
131,小黑盒加速器,
419,大千世界,
750,魔物讨伐团,
1351,星云纪,
1468,太荒初境（试玩版）,
1962,龙魂大冒险,
2140,春秋,
2643,真探,
2657,英雄黄昏-文字版三国志,


Rimuovo queste 14 righe

In [15]:
steam_df = steam_df[steam_df.steam_clean != '']
len(steam_df)

3474

## Matching

### Perfect match between strings

This obtain a score of 100

In [16]:
len(list(set(match_df.twitch_name) & set(steam_df.steam_name)))

318

Aggiorno il dataframe inserendo i 318 match perfetti

In [17]:
for i in range(len(match_df)):
    if match_df.twitch_name[i] in list(steam_df.steam_name):
        match_df.match_score[i] = 100
        match_df.steam_name[i] = match_df.twitch_name[i]
match_df

,twitch_name,twitch_clean,steam_name,steam_clean,match_score
0,Monster Hunter Generations Ultimate,monster hunter generations ultimate,,,
1,Alice: Madness Returns,alice madness returns,Alice: Madness Returns,,100
2,Hold Fast,hold fast,,,
3,Pokémon Trading Card Game Online,pokémon trading card game online,,,
4,Witch It,witch it,Witch It,,100
...,...,...,...,...,...
1139,Rise of Nations: Extended Edition,rise of nations extended edition,Rise of Nations: Extended Edition,,100
1140,Paper Mario: The Origami King,paper mario the origami king,,,
1141,The Inheritance of Crimson Manor,the inheritance of crimson manor,,,
1142,Occult,occult,,,


### Perfect match between cleaned strings

Prima di effettuare il match devo assicurarmi di rimuovere i duplicati all'interno delle liste corrispondenti

In [18]:
import collections
[item for item, count in collections.Counter(list(match_df.twitch_clean)).items() if count > 1]

['f1 2021']

- 'F1 2021' e 'F1® 2021' \
in twitch_names con il cleaning si riducono alla stessa stringa
'f1 2021' \
quindi elimino duplicato.

In [19]:
match_df[match_df.twitch_clean=='f1 2021']

,twitch_name,twitch_clean,steam_name,steam_clean,match_score
121,F1 2021,f1 2021,,,
482,F1® 2021,f1 2021,F1® 2021,,100


In [20]:
dupl = match_df[match_df.twitch_name == 'F1 2021']

Mantengo il duplicato che è già stato matchato

In [21]:
match_df = match_df.drop_duplicates(subset=['twitch_clean'], keep='last').reset_index(drop=True)

In [22]:
match_df[match_df.twitch_clean=='f1 2021']

,twitch_name,twitch_clean,steam_name,steam_clean,match_score
481,F1® 2021,f1 2021,F1® 2021,,100


In [23]:
len(steam_df.steam_clean) - len(set(steam_df.steam_clean))

6

Ci sono 6 duplicati nella lista steam_clean: rimuovo righe associate dal dataframe

In [24]:
steam_df = steam_df.drop_duplicates(subset=["steam_clean"], keep='first').reset_index(drop=True)
steam_df

,steam_name,steam_clean
0,EA SPORTS™ FIFA 21,ea sports fifa 21
1,RoBoRumble,roborumble
2,Anomaly 2,anomaly 2
3,Stealth Inc 2: A Game of Clones,stealth inc 2 a game of clones
4,Banished,banished
...,...,...
3463,Tyranny,tyranny
3464,Viscera Cleanup Detail: Shadow Warrior,viscera cleanup detail shadow warrior
3465,Truck Mechanic: Dangerous Paths - Prologue,truck mechanic dangerous paths prologue
3466,Door Kickers 2: Task Force North,door kickers 2 task force north


In [25]:
len(list(set(match_df.twitch_clean) & set(steam_df.steam_clean)))

402

Ci sono 84 nuovi match tra stringhe pulite: aggiorno il dataframe *match_df*

In [26]:
for i in range(len(match_df)):
    if (match_df.match_score[i] != 100) and (match_df.twitch_clean[i] in list(steam_df.steam_clean)):
        match_df.steam_clean[i] = match_df.twitch_clean[i]
        match_df.match_score[i] = 100

In [27]:
match_df.head(20)

,twitch_name,twitch_clean,steam_name,steam_clean,match_score
0,Monster Hunter Generations Ultimate,monster hunter generations ultimate,,,
1,Alice: Madness Returns,alice madness returns,Alice: Madness Returns,,100
2,Hold Fast,hold fast,,,
3,Pokémon Trading Card Game Online,pokémon trading card game online,,,
4,Witch It,witch it,Witch It,,100
5,Dante's Inferno,dante's inferno,,,
6,The Stanley Parable: Ultra Deluxe,the stanley parable ultra deluxe,The Stanley Parable: Ultra Deluxe,,100
7,The Long Drive,the long drive,The Long Drive,,100
8,Cyber Hunter,cyber hunter,,,
9,Rogue Lords,rogue lords,,,


In [28]:
############ per risalire a steam_name
for i in range(len(match_df)):
    if (match_df.steam_clean[i] != ''):
        index = steam_df.loc[steam_df['steam_clean'] == match_df.steam_clean[i]].index[0]
        match_df.steam_name[i] = steam_df.steam_name[index]
match_df

,twitch_name,twitch_clean,steam_name,steam_clean,match_score
0,Monster Hunter Generations Ultimate,monster hunter generations ultimate,,,
1,Alice: Madness Returns,alice madness returns,Alice: Madness Returns,,100
2,Hold Fast,hold fast,,,
3,Pokémon Trading Card Game Online,pokémon trading card game online,,,
4,Witch It,witch it,Witch It,,100
...,...,...,...,...,...
1138,Rise of Nations: Extended Edition,rise of nations extended edition,Rise of Nations: Extended Edition,,100
1139,Paper Mario: The Origami King,paper mario the origami king,,,
1140,The Inheritance of Crimson Manor,the inheritance of crimson manor,,,
1141,Occult,occult,,,


In [29]:
match_100 = match_df[match_df.match_score==100]

In [30]:
len(match_df) - len(match_100)

741

### Fuzzy match between cleaned strings

with subsets matching starts of the strings

ancora 741 elementi da matchare ( uno non matcha)

In [31]:
match_df = match_df[match_df.match_score != 100].reset_index(drop=True)
len(match_df)

741

In [32]:
for elem in match_df.twitch_name:
    if len(elem) < 4:
        print(elem)

Saw
Art


In [33]:
for elem in steam_df.steam_clean:
    if elem.startswith('saw'):
        print(elem)
    if elem.startswith('art'):
        print(elem)   

artifact


Non ci sono match con 'Saw' e 'Art' di twitch.
Considero i subset di stringhe in steam_df.steam_clean con primi 4 elementi matchati

In [34]:
match_dict = match_df.to_dict('index')
match_dict

{0: {'twitch_name': 'Monster Hunter Generations Ultimate',
  'twitch_clean': 'monster hunter generations ultimate',
  'steam_name': '',
  'steam_clean': '',
  'match_score': ''},
 1: {'twitch_name': 'Hold Fast',
  'twitch_clean': 'hold fast',
  'steam_name': '',
  'steam_clean': '',
  'match_score': ''},
 2: {'twitch_name': 'Pokémon Trading Card Game Online',
  'twitch_clean': 'pokémon trading card game online',
  'steam_name': '',
  'steam_clean': '',
  'match_score': ''},
 3: {'twitch_name': "Dante's Inferno",
  'twitch_clean': "dante's inferno",
  'steam_name': '',
  'steam_clean': '',
  'match_score': ''},
 4: {'twitch_name': 'Cyber Hunter',
  'twitch_clean': 'cyber hunter',
  'steam_name': '',
  'steam_clean': '',
  'match_score': ''},
 5: {'twitch_name': 'Rogue Lords',
  'twitch_clean': 'rogue lords',
  'steam_name': '',
  'steam_clean': '',
  'match_score': ''},
 6: {'twitch_name': "Assassin's Creed IV: Black Flag",
  'twitch_clean': "assassin's creed iv black flag",
  'steam_na

In [35]:
for i in range(len(match_dict)):
    start = match_dict[i]['twitch_clean'][0:4]
    subset = []
    for elem in steam_df.steam_clean:
        if elem.startswith(start):
            subset.append(elem)
    if subset != []:
        score = {}  #key:nome_gioco / value: fuzz.ratio_score
        for elem in subset:
            score[elem] = fuzz.ratio(match_dict[i]['twitch_clean'], elem)
            keys = [k for k, v in score.items() if v == max(score.values())]
            number = list(set([v for k, v in score.items() if v == max(score.values())]))
            match_dict[i]['match_score'] = number[0]
            match_dict[i]['steam_clean'] = keys[0]

In [36]:
nuovo = pd.DataFrame.from_dict(match_dict, orient='index')

############ per risalire a steam_name
for i in range(len(nuovo)):
    if (nuovo.steam_clean[i] != ''):
        index = steam_df.loc[steam_df['steam_clean'] == nuovo.steam_clean[i]].index[0]
        nuovo.steam_name[i] = steam_df.steam_name[index]

In [37]:
fuzzy_match = nuovo[(nuovo.match_score != '')].sort_values(by=['match_score'], ascending=False).reset_index(drop=True)
fuzzy_match

,twitch_name,twitch_clean,steam_name,steam_clean,match_score
0,Monkey Island 2 Special Edition: LeChuck's Rev...,monkey island 2 special edition lechuck's revenge,Monkey Island™ 2 Special Edition: LeChuck’s Re...,monkey island 2 special edition lechucks revenge,99
1,Assassin's Creed IV: Black Flag,assassin's creed iv black flag,Assassin’s Creed® IV Black Flag™,assassins creed iv black flag,98
2,BlazBlue: Central Fiction,blazblue central fiction,BlazBlue Centralfiction,blazblue centralfiction,98
3,Sid Meier's Civilization VI,sid meier's civilization vi,Sid Meier's Civilization® V,sid meier's civilization v,98
4,Assassin's Creed: Brotherhood,assassin's creed brotherhood,Assassin’s Creed® Brotherhood,assassins creed brotherhood,98
...,...,...,...,...,...
468,Xenoverse: Per Aspera Ad Astra,xenoverse per aspera ad astra,Xenobox VR,xenobox vr,31
469,Holdfast: Nations At War,holdfast nations at war,Hold your houses,hold your houses,31
470,WWE SLAM: Card Trader,wwe slam card trader,WWE 2K22,wwe 2k22,29
471,Xenoblade Chronicles: Definitive Edition,xenoblade chronicles definitive edition,Xenobox VR,xenobox vr,29


In [38]:
len(fuzzy_match)

473

In [39]:
no_match = nuovo[(nuovo.match_score == '')]
len(no_match)

268

### Quality assessment and results

- match_score 100 --> inserisco tutte le informazioni

Controllo qualità dei fuzzy con score tra 80 e 100

In [40]:
fuzzy_match[fuzzy_match.match_score.between(80,100)][['twitch_name','steam_name','match_score']]

,twitch_name,steam_name,match_score
0,Monkey Island 2 Special Edition: LeChuck's Rev...,Monkey Island™ 2 Special Edition: LeChuck’s Re...,99
1,Assassin's Creed IV: Black Flag,Assassin’s Creed® IV Black Flag™,98
2,BlazBlue: Central Fiction,BlazBlue Centralfiction,98
3,Sid Meier's Civilization VI,Sid Meier's Civilization® V,98
4,Assassin's Creed: Brotherhood,Assassin’s Creed® Brotherhood,98
...,...,...,...
73,Speedrun,SpeedRunners,80
74,DOOM 3,DOOM,80
75,The Dark Pictures Anthology: Man of Medan,The Dark Pictures Anthology: House of Ashes,80
76,Kingdom Hearts,Kingdom Heroes 8,80


Controllando manualmente la qualità dei match con score tra 80 e 100 otteniamo il seguente subset con 11 *twitch/steam_name* relativi allo stesso gioco: modifichiamo score con 100 e concateniamo

In [41]:
nuovi_100 = fuzzy_match.loc[[0,1,2,3,4,7,23,25,28,61,71]]

Elimino le righe dal df fuzzy_match

In [42]:
fuzzy_match.drop([0,1,2,3,4,7,23,25,28,61,71], axis=0, inplace=True)

In [43]:
len(fuzzy_match)

462

In [44]:
nuovi_100['match_score'] = 100
nuovi_100

,twitch_name,twitch_clean,steam_name,steam_clean,match_score
0,Monkey Island 2 Special Edition: LeChuck's Rev...,monkey island 2 special edition lechuck's revenge,Monkey Island™ 2 Special Edition: LeChuck’s Re...,monkey island 2 special edition lechucks revenge,100
1,Assassin's Creed IV: Black Flag,assassin's creed iv black flag,Assassin’s Creed® IV Black Flag™,assassins creed iv black flag,100
2,BlazBlue: Central Fiction,blazblue central fiction,BlazBlue Centralfiction,blazblue centralfiction,100
3,Sid Meier's Civilization VI,sid meier's civilization vi,Sid Meier's Civilization® V,sid meier's civilization v,100
4,Assassin's Creed: Brotherhood,assassin's creed brotherhood,Assassin’s Creed® Brotherhood,assassins creed brotherhood,100
7,Puyo Puyo Tetris,puyo puyo tetris,Puyo Puyo™Tetris®,puyo puyotetris,100
23,Tom Clancy's The Division 2,tom clancy's the division 2,Tom Clancy’s The Division™,tom clancys the division,100
25,Infernax,infernax,Inferna,inferna,100
28,Dark Messiah of Might and Magic,dark messiah of might and magic,Dark Messiah of Might & Magic,dark messiah of might magic,100
61,Friday the 13th: The Computer Game,friday the 13th the computer game,Friday the 13th: The Game,friday the 13th the game,100


In [45]:
match_100 = pd.concat([match_100,nuovi_100]).reset_index(drop=True)

Sempre nei match con score tra 80 e 100 otteniamo il seguente subset con 8 twitch/steam_name relativi giochi diversi: eliminiamo lo score

In [46]:
nuovi_no_match = fuzzy_match.loc[[27,40,56,62,67,69,74,76,77]]

Elimino le righe dal df fuzzy_match

In [47]:
fuzzy_match.drop([27,40,56,62,67,69,74,76,77], axis=0, inplace=True)
len(fuzzy_match)

453

In [48]:
nuovi_no_match['match_score'] = ''
nuovi_no_match

,twitch_name,twitch_clean,steam_name,steam_clean,match_score
27,Stronghold Crusader II,stronghold crusader ii,Stronghold Crusader 2,stronghold crusader 2,
40,BUS SIMULATOR,bus simulator,Bus Simulator 16,bus simulator 16,
56,FIFA 21,fifa 21,FIFA 22,fifa 22,
62,Expedition Zero,expedition zero,Expeditions: Rome,expeditions rome,
67,The King of Fighters All-Star,the king of fighters allstar,THE KING OF FIGHTERS XV,the king of fighters xv,
69,Tomb Raider: Gold,tomb raider gold,Tomb Raider,tomb raider,
74,DOOM 3,doom 3,DOOM,doom,
76,Kingdom Hearts,kingdom hearts,Kingdom Heroes 8,kingdom heroes 8,
77,Virtual Casino,virtual casino,VirtualCast,virtualcast,


In [49]:
no_match = pd.concat([no_match,nuovi_no_match]).reset_index(drop=True)
len(no_match)

277

#### Results

In [50]:
len(match_100) + len(fuzzy_match) + len(no_match) + len(dupl)

1144

In [51]:
games_total = pd.concat([match_100,fuzzy_match,no_match,dupl]).reset_index(drop=True)
games_total

,twitch_name,twitch_clean,steam_name,steam_clean,match_score
0,Alice: Madness Returns,alice madness returns,Alice: Madness Returns,,100
1,Witch It,witch it,Witch It,,100
2,The Stanley Parable: Ultra Deluxe,the stanley parable ultra deluxe,The Stanley Parable: Ultra Deluxe,,100
3,The Long Drive,the long drive,The Long Drive,,100
4,INSIDE,inside,INSIDE,,100
...,...,...,...,...,...
1139,Tomb Raider: Gold,tomb raider gold,Tomb Raider,tomb raider,
1140,DOOM 3,doom 3,DOOM,doom,
1141,Kingdom Hearts,kingdom hearts,Kingdom Heroes 8,kingdom heroes 8,
1142,Virtual Casino,virtual casino,VirtualCast,virtualcast,


In [52]:
#games_total.to_csv('games_total.csv')

Dall'analisi dei fuzzy_match con score maggiore di 70, estriamo le righe relative a match maggiori o uguali a score 79, perchè relative a match tra edizioni differenti del medesimo videogioco: per questo motivo questi match corrispondono ad un match di categorie di gioco

In [53]:
fuzzy_high = fuzzy_match[fuzzy_match.match_score.between(79,100)]
len(fuzzy_high)

66

## Merging

- match_100 : enrichment con 'title','Developer','Publisher','Release Date','categories';

- fuzzy_high : enrichment con 'title','Developer','Publisher','categories'

controllo i 10 giochi duplicati

In [54]:
for row in range(len(games_total)):
    if games_total.steam_name[row] in dupl_games:
        print(row)

45


In [55]:
dupl_games

["Conqueror's Blade",
 'DEAD OR ALIVE Xtreme Venus Vacation',
 'Fallout: New Vegas',
 'Ironsight',
 'Romance of the Three Kingdoms Hadou',
 'Wolfenstein II: The New Colossus',
 'Dishonored',
 'Wolfenstein: The New Order',
 'Anomaly Zone',
 'GRID']

In [56]:
for row in range(len(games_total)):
    if games_total.steam_name[row] in dupl_games:
        print(games_total.twitch_name[row])

Fallout: New Vegas


Tra i giochi per cui faremo enrichment, 'Fallout: New Vegas' presenta un duplicato in steam_db

In [57]:
for key in games:
    if games[key]['title'] == 'Fallout: New Vegas':
        print(key,games[key])

22380 {'title': 'Fallout: New Vegas', 'Developer': 'Obsidian Entertainment', 'Publisher': 'Bethesda Softworks', 'Release Date': '19 October 2010', '24h Player Peak': '5,049', 'Followers': '143,057', 'categories': ['Open World', 'RPG', 'Post-apocalyptic', 'Singleplayer', 'Moddable', 'Exploration']}
22490 {'title': 'Fallout: New Vegas', 'Developer': 'Obsidian Entertainment', 'Publisher': 'Bethesda Softworks', 'Release Date': '21 October 2010', '24h Player Peak': '817', 'Followers': '9,254', 'categories': ['RPG', 'Post-apocalyptic', 'Open World', 'Atmospheric', 'Story Rich', 'Action']}


Scelgo di arricchirre con le info del gioco associato alla data di rilascio '19 october 2010' [22380], con maggiori followers:
il successivo [22490], data di rilascio due giorni dopo, è probabilmente una patch.    

In [58]:
to_enrich = pd.concat([match_100,fuzzy_high])
to_enrich

,twitch_name,twitch_clean,steam_name,steam_clean,match_score
0,Alice: Madness Returns,alice madness returns,Alice: Madness Returns,,100
1,Witch It,witch it,Witch It,,100
2,The Stanley Parable: Ultra Deluxe,the stanley parable ultra deluxe,The Stanley Parable: Ultra Deluxe,,100
3,The Long Drive,the long drive,The Long Drive,,100
4,INSIDE,inside,INSIDE,,100
...,...,...,...,...,...
81,Dragon Ball Z,dragon ball z,DRAGON BALL FighterZ,dragon ball fighterz,79
82,Assassin's Creed Valhalla,assassin's creed valhalla,Assassin's Creed 2,assassin's creed 2,79
83,Dissidia Final Fantasy NT,dissidia final fantasy nt,DISSIDIA FINAL FANTASY NT Free Edition,dissidia final fantasy nt free edition,79
84,Call of Duty: Mobile,call of duty mobile,Call of Duty® 2,call of duty 2,79


In [59]:
games_tw = games_tw.rename(columns={'game_name':'twitch_name'})
games_tw

,twitch_name
ID_game,
0,Monster Hunter Generations Ultimate
1,Alice: Madness Returns
2,Hold Fast
3,Pokémon Trading Card Game Online
4,Witch It
...,...
1139,Rise of Nations: Extended Edition
1140,Paper Mario: The Origami King
1141,The Inheritance of Crimson Manor


In [60]:
info_df = games_tw.merge(to_enrich[['twitch_name','steam_name','match_score']] , on= 'twitch_name', how='left')
info_df['Developer'] = ''
info_df['Publisher'] = ''
info_df['Release_Date'] = ''
info_df['categories'] = ''
info_df

,twitch_name,steam_name,match_score,Developer,Publisher,Release_Date,categories
0,Monster Hunter Generations Ultimate,NaN,NaN,,,,
1,Alice: Madness Returns,Alice: Madness Returns,100,,,,
2,Hold Fast,NaN,NaN,,,,
3,Pokémon Trading Card Game Online,NaN,NaN,,,,
4,Witch It,Witch It,100,,,,
...,...,...,...,...,...,...,...
1139,Rise of Nations: Extended Edition,Rise of Nations: Extended Edition,100,,,,
1140,Paper Mario: The Origami King,NaN,NaN,,,,
1141,The Inheritance of Crimson Manor,NaN,NaN,,,,
1142,Occult,NaN,NaN,,,,


In [61]:
games_df = pd.DataFrame.from_dict(games,orient='index')
games_df

,title,Developer,Publisher,Release Date,Last Update,24h Player Peak,Followers,categories
1599340,Lost Ark,Smilegate RPG,Amazon Games,11 February 2022,9 June 2022,"880,727","268,276","[MMORPG, Free to Play, Action RPG, RPG, Action..."
730,Counter-Strike: Global Offensive,Valve,NaN,21 August 2012,2 June 2022,"840,048","3,155,868","[FPS, Shooter, Multiplayer, Competitive, Actio..."
570,Dota 2,Valve,NaN,9 July 2013,13 June 2022,"672,736","1,981,324","[Free to Play, MOBA, Multiplayer, Strategy, eS..."
578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.",NaN,21 December 2017,9 June 2022,"367,516","3,032,121","[Survival, Shooter, Multiplayer, Battle Royale..."
1172470,Apex Legends™,Respawn Entertainment,Electronic Arts,4 November 2020,7 June 2022,"302,819","323,408","[Free to Play, Battle Royale, Multiplayer, Sho..."
...,...,...,...,...,...,...,...,...
220820,Zombie Driver HD,Exor Studios,NaN,17 October 2012,NaN,9,"6,586","[Zombies, Racing, Top-Down, Gore, Violent, Ope..."
491030,The Last Error,Anatoliy Loginovskikh,NaN,Unknown,NaN,0,108,"[Action, 2D, Puzzle, Shooter]"
370510,Marble Mayhem: Fragile Ball,"Bimboosoft Co.,Ltd.",Senpai Studios,7 July 2015,NaN,1,696,"[Casual, Indie, Puzzle, Difficult, Arcade, 2D]"
524680,Escape From Nazi Labs,Manic Mind Game Lab,NaN,Unknown,NaN,0,325,"[Action, RPG, Indie, Perma Death, Retro, Proce..."


Aggiungo le informazioni di 'Fallout: New Vegas' relative alla chiave '22380', escludendo la patch successiva

In [62]:
for i in range(len(info_df)):
    if info_df.twitch_name[i] == 'Fallout: New Vegas': #22380
        info_df.Developer[i] = games_df['Developer']['22380']
        info_df.Publisher[i] = games_df['Publisher']['22380']
        info_df.Release_Date[i] = games_df['Release Date']['22380']
        info_df.Developer[i] = games_df['Developer']['22380']
        info_df.categories[i] = games_df['categories']['22380']

In [63]:
info_df.columns

Index(['twitch_name', 'steam_name', 'match_score', 'Developer', 'Publisher',
       'Release_Date', 'categories'],
      dtype='object')

Per match perfetti con score 100 inserisco tutte le informazioni presenti nel json

In [64]:
for i in range(len(info_df)):
    if (info_df.twitch_name[i] != 'Fallout: New Vegas') & (info_df.match_score[i] == 100):
        index = games_df.loc[games_df['title'] == info_df.steam_name[i]].index[0]
        info_df.Developer[i] = games_df['Developer'][index]
        info_df.Publisher[i] = games_df['Publisher'][index]
        info_df.Release_Date[i] = games_df['Release Date'][index]
        info_df.categories[i] = games_df['categories'][index]

In [65]:
info_df['match_score'] = info_df['match_score'].fillna(0)

Per match con score tra 79 e 99 inserisco le informazioni 'developer','publisher','categories' presenti nel json

In [66]:
for i in range(len(info_df)):
    if (info_df.twitch_name[i] != 'Fallout: New Vegas') & (79 <= info_df['match_score'][i] <= 99):
        index = games_df.loc[games_df['title'] == info_df.steam_name[i]].index[0]
        info_df.Developer[i] = games_df['Developer'][index]
        info_df.Publisher[i] = games_df['Publisher'][index]
        info_df.categories[i] = games_df['categories'][index]

C:\Users\gianl\AppData\Local\Temp/ipykernel_12756/779521762.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_df.Developer[i] = games_df['Developer'][index]
C:\Users\gianl\AppData\Local\Temp/ipykernel_12756/779521762.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_df.Publisher[i] = games_df['Publisher'][index]
C:\Users\gianl\AppData\Local\Temp/ipykernel_12756/779521762.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [67]:
info_df['Release_Date'] = info_df['Release_Date'].replace(['Unknown', ''], np.nan, regex=True)
info_df

,twitch_name,steam_name,match_score,Developer,Publisher,Release_Date,categories
0,Monster Hunter Generations Ultimate,NaN,0,,,NaN,
1,Alice: Madness Returns,Alice: Madness Returns,100,Spicy Horse Games,Electronic Arts,17 June 2011,"[3D Platformer, Action-Adventure, Hack and Sla..."
2,Hold Fast,NaN,0,,,NaN,
3,Pokémon Trading Card Game Online,NaN,0,,,NaN,
4,Witch It,Witch It,100,Barrel Roll Games,Daedalic Entertainment,22 October 2020,"[Multiplayer, Funny, Casual, Action, Indie, Hi..."
...,...,...,...,...,...,...,...
1139,Rise of Nations: Extended Edition,Rise of Nations: Extended Edition,100,SkyBox Labs,Xbox Game Studios,12 June 2014,"[Strategy, RTS, Classic, Multiplayer, Historic..."
1140,Paper Mario: The Origami King,NaN,0,,,NaN,
1141,The Inheritance of Crimson Manor,NaN,0,,,NaN,
1142,Occult,NaN,0,,,NaN,


In [68]:
#info_df = info_df.replace(np.nan, '', regex=True)
info_df.drop(['match_score'], axis = 1, inplace = True)

In [69]:
info_df['Release_Date'] = pd.to_datetime(info_df['Release_Date'], format='%d %B %Y')
info_df

,twitch_name,steam_name,Developer,Publisher,Release_Date,categories
0,Monster Hunter Generations Ultimate,NaN,,,NaT,
1,Alice: Madness Returns,Alice: Madness Returns,Spicy Horse Games,Electronic Arts,2011-06-17,"[3D Platformer, Action-Adventure, Hack and Sla..."
2,Hold Fast,NaN,,,NaT,
3,Pokémon Trading Card Game Online,NaN,,,NaT,
4,Witch It,Witch It,Barrel Roll Games,Daedalic Entertainment,2020-10-22,"[Multiplayer, Funny, Casual, Action, Indie, Hi..."
...,...,...,...,...,...,...
1139,Rise of Nations: Extended Edition,Rise of Nations: Extended Edition,SkyBox Labs,Xbox Game Studios,2014-06-12,"[Strategy, RTS, Classic, Multiplayer, Historic..."
1140,Paper Mario: The Origami King,NaN,,,NaT,
1141,The Inheritance of Crimson Manor,NaN,,,NaT,
1142,Occult,NaN,,,NaT,


In [70]:
info_df.index.name = 'idGame'
info_df.columns = ['gameName', 'steamName', 'developer', 'publisher', 'releaseDate', 'categories']
info_df

,gameName,steamName,developer,publisher,releaseDate,categories
idGame,,,,,,
0,Monster Hunter Generations Ultimate,NaN,,,NaT,
1,Alice: Madness Returns,Alice: Madness Returns,Spicy Horse Games,Electronic Arts,2011-06-17,"[3D Platformer, Action-Adventure, Hack and Sla..."
2,Hold Fast,NaN,,,NaT,
3,Pokémon Trading Card Game Online,NaN,,,NaT,
4,Witch It,Witch It,Barrel Roll Games,Daedalic Entertainment,2020-10-22,"[Multiplayer, Funny, Casual, Action, Indie, Hi..."
...,...,...,...,...,...,...
1139,Rise of Nations: Extended Edition,Rise of Nations: Extended Edition,SkyBox Labs,Xbox Game Studios,2014-06-12,"[Strategy, RTS, Classic, Multiplayer, Historic..."
1140,Paper Mario: The Origami King,NaN,,,NaT,
1141,The Inheritance of Crimson Manor,NaN,,,NaT,


In [71]:
info_df['categories']=info_df['categories'].astype(str)
info_df['categories']=info_df['categories'].replace(["\[","\]","'"], "",regex=True)
info_df['categories']=info_df['categories'].replace(',', ';',regex=True)
info_df

,gameName,steamName,developer,publisher,releaseDate,categories
idGame,,,,,,
0,Monster Hunter Generations Ultimate,NaN,,,NaT,
1,Alice: Madness Returns,Alice: Madness Returns,Spicy Horse Games,Electronic Arts,2011-06-17,3D Platformer; Action-Adventure; Hack and Slas...
2,Hold Fast,NaN,,,NaT,
3,Pokémon Trading Card Game Online,NaN,,,NaT,
4,Witch It,Witch It,Barrel Roll Games,Daedalic Entertainment,2020-10-22,Multiplayer; Funny; Casual; Action; Indie; Hid...
...,...,...,...,...,...,...
1139,Rise of Nations: Extended Edition,Rise of Nations: Extended Edition,SkyBox Labs,Xbox Game Studios,2014-06-12,Strategy; RTS; Classic; Multiplayer; Historica...
1140,Paper Mario: The Origami King,NaN,,,NaT,
1141,The Inheritance of Crimson Manor,NaN,,,NaT,


In [72]:
info_df.to_csv('Output_datasets/games_dataset.csv')